## Análise de Sentimentos

In [2]:
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", device=0)
result = sentiment_analysis("Esse produto é maravilhoso!")
print(result)


[{'label': '5 stars', 'score': 0.8916902542114258}]


O modelo `nlptown/bert-base-multilingual-uncased-sentiment` é ajustado para **análise de sentimentos em 5 níveis**, com as seguintes labels:

- **1 star**: Sentimento muito negativo
- **2 stars**: Sentimento negativo
- **3 stars**: Sentimento neutro
- **4 stars**: Sentimento positivo
- **5 stars**: Sentimento muito positivo


## NER

In [106]:
from transformers import pipeline

# Configura o pipeline para classificação de token com truncamento
ner = pipeline("token-classification", model="pierreguillou/ner-bert-base-cased-pt-lenerbr", device=0)
entities = ner("O presidente da França é Emmanuel Macron. E a chanceler da Alemanha é Angela Merkel. Mas minha casa é em São Paulo.")
print(entities)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}, {'entity': 'B-PESSOA', 'score': np.float32(0.9997147), 'index': 6, 'word': 'Emma', 'start': 25, 'end': 29}, {'entity': 'B-PESSOA', 'score': np.float32(0.99970573), 'index': 7, 'word': '##nu', 'start': 29, 'end': 31}, {'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 8, 'word': '##el', 'start': 31, 'end': 33}, {'entity': 'I-PESSOA', 'score': np.float32(0.9997819), 'index': 9, 'word': 'Mac', 'start': 34, 'end': 37}, {'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 10, 'word': '##ron', 'start': 37, 'end': 40}, {'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}, {'entity': 'B-PESSOA', 'score': np.float32(0.9995801), 'index': 18, 'word': 'Angel', 'start': 70, 'end': 75}, {'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 19, 'word': '##a', 'start': 75, 'end': 76}, {'entity': 'I-PE

In [107]:
from typing import List, Dict
import numpy as np

def group_entities(entities: List[Dict]) -> List[Dict]:
    grouped_entities = []
    current_group = None

    for entity in entities:
        if current_group and entity['start'] == current_group['end']:
            # Append to the current group
            current_group['word'] += entity['word'].replace("##", "")
            current_group['end'] = entity['end']
            current_group['score'] = min(current_group['score'], entity['score'])  # Atualiza o score
        else:
            # Finaliza o grupo atual e inicia um novo
            if current_group:
                grouped_entities.append(current_group)
            current_group = entity.copy()  # Inicia um novo grupo com a entidade atual
    
    # Adiciona o último grupo
    if current_group:
        grouped_entities.append(current_group)

    return grouped_entities

In [108]:
result = group_entities(entities)

# Resultado agrupado
for item in result:
    print(item)

{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}
{'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 6, 'word': 'Emmanuel', 'start': 25, 'end': 33}
{'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 9, 'word': 'Macron', 'start': 34, 'end': 40}
{'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}
{'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 18, 'word': 'Angela', 'start': 70, 'end': 76}
{'entity': 'I-PESSOA', 'score': np.float32(0.9997882), 'index': 20, 'word': 'Merkel', 'start': 77, 'end': 83}
{'entity': 'B-LOCAL', 'score': np.float32(0.9991271), 'index': 28, 'word': 'São', 'start': 105, 'end': 108}
{'entity': 'I-LOCAL', 'score': np.float32(0.99898416), 'index': 29, 'word': 'Paulo', 'start': 109, 'end': 114}


In [120]:
final = []

for pos, item in enumerate(result):
    if 'B' in item['entity']:
        new_group = []
        new_group.append(item)
        i = 1
        while 'I' in result[result.index(item)+i]['entity']:
            if pos+i+1 < len(result):
                new_group.append(result[result.index(item)+i])
                i += 1
            else:
                new_group.append(result[result.index(item)+i])
                break
    if 'I' in item['entity']:
        continue
    print(new_group)

[{'entity': 'B-LOCAL', 'score': np.float32(0.6032972), 'index': 4, 'word': 'França', 'start': 16, 'end': 22}]
[{'entity': 'B-PESSOA', 'score': np.float32(0.9997003), 'index': 6, 'word': 'Emmanuel', 'start': 25, 'end': 33}, {'entity': 'I-PESSOA', 'score': np.float32(0.99977344), 'index': 9, 'word': 'Macron', 'start': 34, 'end': 40}]
[{'entity': 'B-LOCAL', 'score': np.float32(0.73137456), 'index': 16, 'word': 'Alemanha', 'start': 59, 'end': 67}]
[{'entity': 'B-PESSOA', 'score': np.float32(0.9995579), 'index': 18, 'word': 'Angela', 'start': 70, 'end': 76}, {'entity': 'I-PESSOA', 'score': np.float32(0.9997882), 'index': 20, 'word': 'Merkel', 'start': 77, 'end': 83}]
[{'entity': 'B-LOCAL', 'score': np.float32(0.9991271), 'index': 28, 'word': 'São', 'start': 105, 'end': 108}, {'entity': 'I-LOCAL', 'score': np.float32(0.99898416), 'index': 29, 'word': 'Paulo', 'start': 109, 'end': 114}]


## Sumarização 

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Carrega o tokenizer e o modelo na GPU
tokenizer = T5Tokenizer.from_pretrained("PamelaBorelli/flan-t5-base-summarization-pt-br")
# model = T5ForConditionalGeneration.from_pretrained("PamelaBorelli/flan-t5-base-summarization-pt-br").to("cuda")
model = T5ForConditionalGeneration.from_pretrained("PamelaBorelli/flan-t5-base-summarization-pt-br")

# Texto de entrada e tokenização
input_text = (
    "O corpo está mais propenso a sentir dores com exercícios de alta intensidade | Foto: Getty Images O problema está em saber identificar qual é qual. "
    "'Em algumas situações, é difícil diferenciar uma da outra', reconhece Juan Francisco Marco, professor do Centro de Ciência do Esporte, Treinamento e Fitness Alto Rendimento, na Espanha. "
    "'A dor boa é aquela que associamos ao exercício físico, que não limita (o movimento) e permite continuar (a se exercitar) até o momento em que o músculo fica realmente esgotado e não trabalha mais', explica. "
    "É importante detectar qual é o tipo de dor que você está sentindo, para evitar ter problemas mais sérios | Foto: Getty Images Para Francisco Sánchez Diego, diretor do centro de treinamento Corpore 10, 'a dor boa se sente no grupo muscular que você trabalhou, tanto durante o treinamento como nos dias seguintes'."
)

# Move os input_ids para a GPU
#input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# Geração do resumo
outputs = model.generate(input_ids, max_new_tokens=250, min_length=25)

# Decodificação e impressão do resultado
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

O problema é difcil diferenciar uma da outra. 'A dor boa é aquela que associamos ao exerccio fsico, que no limita (o movimento) e permite continuar (a se exercitar) até o moment em que o msculo fica realmente esgotado e no trabalha mais', explica Juan Francisco Marco.


In [9]:
# Texto de entrada e tokenização
input_text = "Tecnologia é o conjunto de processos e habilidades usados na produção de bens ou serviços, ou na realização de objetivos, como em investigações científicas. Isso também pode ser embutido em máquinas para permitir a operação destas sem conhecimento detalhado do funcionamento. Sistema tecnológico (ou computacional) que através do input de dados, faz à análise destes e, então produz um resultado. A forma mais simples de tecnologia é o desenvolvimento e a utilização de ferramentas básicas. A descoberta pré-histórica de como controlar o fogo e a subsequente Revolução Neolítica aumentaram a disponibilidade de fontes de alimento, enquanto a invenção da roda auxiliou humanos a viajar, transportar cargas e controlar seu ambiente. Desenvolvimentos ao longo da história, como a prensa móvel, também conhecida como impressora, o telefone e a Internet diminuíram as barreiras físicas da comunicação e permitiram que os humanos interagissem livremente em uma escala global."

# Move os input_ids para a GPU
#input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
# Geração do resumo
outputs = model.generate(input_ids, max_new_tokens=250, min_length=25)

# Decodificação e impressão do resultado
print(tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False))

A tecnologia é o conjunto de processos e habilidades usados na produço de bens ou serviços ou na realizaço de objetivos, como em investigaçes cientficas. A forma mais simples de tecnologia é o desenvolvimento e a utilizaço de ferramentas básicas.


## Pergunta e Resposta

In [13]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="pierreguillou/bert-base-cased-squad-v1.1-portuguese")
context = "O Brasil é o maior país da América do Sul e faz fronteira com quase todos os países da região."
result = question_answerer(question="Qual é o maior país da América do Sul?", context=context)
print(result)


config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


{'score': 0.9634592533111572, 'start': 2, 'end': 8, 'answer': 'Brasil'}


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [20]:
result = question_answerer(question="Qual é a cor do cavalo branco de napoleão ?", context='Napoleão tinha um cavalo marrom')
print(result)

{'score': 0.9968142509460449, 'start': 25, 'end': 31, 'answer': 'marrom'}


## Geração de Texto

In [6]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-1B"

pipe = pipeline(
    "text-generation", 
    model=model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto",

)

pipe("A chave para a vida é", max_new_tokens=20)


Some parameters are on the meta device because they were offloaded to the disk.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'A chave para a vida é a comunhão de amor. Uma vez que vocês se amam, a vida é mais'}]

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# caminho_modelo = "zhengr/MixTAO-7Bx2-MoE-Instruct-v7.0"
# caminho_modelo = "mistralai/Mistral-7B-v0.1"
# caminho_modelo = "stabilityai/stablelm-zephyr-3b"
# caminho_modelo = "HuggingFaceH4/zephyr-7b-beta"
caminho_modelo = "stabilityai/stablelm-2-zephyr-1_6b"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carrega tokenizer e modelo uma única vez
tokenizer = AutoTokenizer.from_pretrained(caminho_modelo, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(caminho_modelo, trust_remote_code=True).to(device)

if device.type == 'cuda':
  torch.set_default_tensor_type(torch.cuda.BFloat16Tensor)

# Garantir que pad_token_id esteja definido
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [11]:
prompt = f"""<|im_start|>O segredo da vida é<|im_end|>"""

tokens_entrada = tokenizer.encode(prompt, return_tensors='pt', truncation=True, max_length=16).to(device)
max_length = tokens_entrada.shape[1] + 10  # Ajuste conforme necessário

# Gera a máscara de atenção para todos os tokens
attention_mask = torch.ones(tokens_entrada.shape, dtype=torch.long, device=device)

# Geração do texto
resumos = model.generate(
	input_ids=tokens_entrada,
	attention_mask=attention_mask,
	max_length=max_length,
	do_sample=True,
	temperature=0.1,
	num_return_sequences=1,
	eos_token_id=tokenizer.eos_token_id,
	pad_token_id=tokenizer.pad_token_id
)

texto_saida = tokenizer.decode(resumos[0], skip_special_tokens=True)
num_tokens_saida = tokenizer(texto_saida, return_tensors='pt').input_ids.shape[1]

In [12]:
texto_saida

'O segredo da vida é� encontrar a felicidade em cada momento presente'